In [1]:
from IPython.display import HTML

HTML('''<script>
code_show=true;
function code_toggle() {
    if (code_show){
        $('div.input').hide();
    } else {
        $('div.input').show();
    }
    code_show = !code_show
}
$(document).ready(code_toggle);
</script>
Click <a href="javascript:code_toggle()">here</a> to toggle on/off the code cells.''')



In [2]:
import ipywidgets as widgets
from IPython.display import display
import warnings
import PyPDF2
import datetime
import functools

warnings.filterwarnings('ignore')

# Create the file upload widget
file_upload = widgets.FileUpload(
    accept='',  # Accepted file types (e.g., '.txt', '.pdf', '.jpg', '.csv', etc.); leave empty for all file types
    multiple=True  # Set to True if you want to allow multiple files to be uploaded
)

fileListOutput = widgets.Output()

# Display the widget
display(file_upload, fileListOutput)

ongFileUploadVals = []

def on_value_change(change):
    global ongFileUploadVals
    global file_upload
    
    if (ongFileUploadVals ==  [] and list(file_upload.value) == []) or ongFileUploadVals != list(file_upload.value):

        @fileListOutput.capture()
        def file_list_output_render():
            global ongFileUploadVals
            
            fileListOutput.clear_output()
            display(widgets.Label("Uploaded Files: "))
            
            if change != {}:

                # set file_upload to previous file_upload and current file_upload files
                updatedFileUploadVal = []
                for file in ongFileUploadVals:
                    updatedFileUploadVal.append(file)

                for file in file_upload.value:
                    if file not in updatedFileUploadVal:
                        updatedFileUploadVal.append(file)

                # update ong_file_upload value
                ongFileUploadVals = updatedFileUploadVal
            
                for file in ongFileUploadVals:
                    # extract number of pages of uploaded files
                    
                    fileOpen = open(file['name'], 'rb')
                    reader = PyPDF2.PdfReader(fileOpen)
                    totalPgs = len(reader.pages)
                    remButton = widgets.Button(description='Remove')
                    
                    def remove_button_onclick(button, fileInfo=""):
                        global ongFileUploadVals
                        # update file_upload.value to remove chosen file
                        newVal = []
                        for file in file_upload.value:
                            if file['name'] != fileInfo:
                                newVal.append(file)
                        ongFileUploadVals = []
                        file_upload.value = tuple(newVal)
                        
                    remButton.on_click(functools.partial(remove_button_onclick, fileInfo=file['name']))
                    
                    display(widgets.Label("\n  file name: " + file['name'] + "  #pages: " + str(totalPgs)), remButton)

                    file_upload.value = tuple(updatedFileUploadVal)

        file_list_output_render()

file_upload.observe(on_value_change, names='value')

FileUpload(value=(), description='Upload', multiple=True)

Output()

In [3]:
print(file_upload.value)

()


In [12]:
import io
import PyPDF2
import ipywidgets as widgets
import torch
from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer, AutoModelForQuestionAnswering
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
from tqdm import tqdm

documents = []
embeddings = []

def create_embeddings(documents, model_name="paraphrase-MiniLM-L6-v2"):
    #device = torch.device("cuda" if torch.cuda.is_available() else print("cpu"))
    model = SentenceTransformer(model_name)
    #model.to("cuda")
    embeddings = model.encode(documents)
    
    return embeddings

def split_string(input_string, chunk_size):
    return [input_string[i:i+chunk_size] for i in range(0, len(input_string), chunk_size)]

def upload_file(file_upload, i):
    global documents
    global embeddings

    text_content = ''

    # Get the uploaded file's content
    uploaded_file_data = [f.content.tobytes() for f in file_upload.value]
    
    # Get the uploaded file's content - OLD IPYWIDGETS
    # for file, attributes in file_upload.value.items():
    #     content = attributes.get('content') 
    #     uploaded_file_data.append(content)    
    
    uploaded_file_content = uploaded_file_data[i]
    
    # Convert the uploaded file content to a readable file-like buffer
    file_buffer = io.BytesIO(uploaded_file_content)

    # Read the PDF file using PyPDF2
    pdf_reader = PyPDF2.PdfReader(file_buffer)

    # Extract the text content from the PDF
    text_content = ''
    for page_num in tqdm(range(len(pdf_reader.pages)), desc='Processing pages'):
        page = pdf_reader.pages[page_num]
        text_content += page.extract_text()

    # on last iteration add the information to documents and embeddings
    if page_num == (len(pdf_reader.pages)-1):
        documents.append(split_string(text_content, 1000))
    
        # Create document embeddings for semantic search
        embeddings.append(create_embeddings(documents[i]))        

# for file, attributes in file_upload.value.items():
#     content = attributes.get('content') 
#     uploaded_file_data.append(content)
    
button = widgets.Button(description="Load the data")

def on_button_click(b):
    if file_upload.value:
        for i in range(len(file_upload.value)):
            upload_file(file_upload, i)
            print("Data loaded successfully!")
    else:
        print("No files uploaded")

button.on_click(on_button_click)

display(button)

Button(description='Load the data', style=ButtonStyle())

In [36]:
import torch
from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer, AutoModelForQuestionAnswering
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
from IPython.display import clear_output


from transformers import logging

logging.set_verbosity_error()

def retrieve_passages(query, documents, embeddings, top_k=5):
    model = SentenceTransformer("paraphrase-MiniLM-L6-v2")
    #model.to("cuda")

    query_embedding = model.encode([query])[0]
    similarities = cosine_similarity(query_embedding.reshape(1, -1), embeddings)

    #top_indices = np.argsort(similarities[0])[-top_k:][::-1]

    threshold = 0.4  
    top_indices = np.argsort(similarities[0])[::-1][:top_k][similarities[0][np.argsort(similarities[0])[::-1][:top_k]] >= threshold]

    return [documents[i] for i in top_indices]
    
def extract_answer(question, passage, model_name="roberta-large"):
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForQuestionAnswering.from_pretrained(model_name)

    inputs = tokenizer.encode_plus(question, str(passage), return_tensors="pt", max_length=512, truncation=True, padding='max_length', return_offsets_mapping=True)
    input_ids = inputs["input_ids"]
    attention_mask = inputs["attention_mask"]
    offset_mapping = inputs["offset_mapping"].squeeze(0)

    outputs = model(input_ids, attention_mask=attention_mask)

    answer_start = torch.argmax(outputs.start_logits)
    answer_end = torch.argmax(outputs.end_logits)

    answer_start_char = offset_mapping[answer_start][0]
    answer_end_char = offset_mapping[answer_end][1]
    answer = passage[answer_start_char:answer_end_char]

    return answer

def qa_system(question, documents, embeddings, model_name="roberta-large", top_k=5):
        answers = []
        # keys = list(file_upload.value.keys())
    
        # iterate over all files to extract answers from each one
        for i in range(len(documents)):
            passages = retrieve_passages(question, documents[i], embeddings[i], top_k=top_k)
        
            #print(passages)
        
            for passage in passages:
                answer = extract_answer(question, passage, model_name=model_name)
                answers.append({"fileName" : file_upload.value[i]['name'], "answer": answer, "context": passage})
                # answers.append({"fileName" : file_upload.value[keys[i]]['metadata']['name'], "answer": answer, "context": passage})
                
        return answers

import ipywidgets as widgets
from IPython.display import display
import string

# This function is called when the button is clicked

# Create the text field and button widgets
text_field = widgets.Text(placeholder='Enter your question here')
button = widgets.Button(description='Search')

answers = []

def on_button_click(button):
    
    %clear
    clear_output(wait=True)
    # Get the text input from the text field
    text_input = text_field.value
    # Display the widgets
    text_field.value = text_input
    display(text_field, button)   
    
    out.clear_output()
    print("Generating answers, please wait...")
    # Retrieve answers using the QA system
    global answers
    answers = qa_system(text_input, documents, embeddings)

    # Print the answers
    for idx, answer in enumerate(answers):
        print(f"Document : {answer['fileName']}, Answer {idx + 1}: {answer['answer']}\nContext: {answer['context']}\n") 
     
    if not answers: 
        print("No answers available")
    else:
        show_results()
    
# Set the callback function for the button
button.on_click(on_button_click)

# output to display answers 
out = widgets.Output(layout={'width': '100%', 'height': '100%'})

# Display the widgets
display(text_field, button, out)

# OPTION #1 - OPENS FILE AS URL 

import ipywidgets as widgets
from IPython.display import display, IFrame
import fitz 
import functools
import string
import os, sys, subprocess
import webbrowser
    
fileToClear = ''
def clear_file():
    if fileToClear != '':
        pdfDoc = fitz.open(fileToClear)
    
        # iterates pages and removes annotations generated by fitz
        for pgNum in range(len(pdfDoc)):
            page = pdfDoc[pgNum]
    
            pdfDoc.xref_set_key(page.xref, "Annots", "[]")
        
        pdfDoc.save(fileToClear, incremental=True, encryption=fitz.PDF_ENCRYPT_KEEP)

def clear_btn_click(button): clear_file();

def open_file(filename, pgNum):
    currDir = os.getcwdb()

    # opens file in default browser at specific page, NOTE: assumes that file is in current directory and Chrome is installed
    fileLoc = "file:///" + str(currDir)[2:][:-1] + "/" + filename.replace(" ", "%20") + "#page=" + str(pgNum)

    (webbrowser.get(using='chrome')).open(fileLoc)      
    

# https://bobbyhadz.com/blog/python-remove-non-ascii-characters-from-string
def remove_non_ascii(txt):
    ascii_chars = set(string.printable)

    return ''.join(
        filter(lambda x: x in ascii_chars, txt)
   )

# https://www.geeksforgeeks.org/python-remove-punctuation-from-string/
def remove_punct(txt):
    return txt.translate(str.maketrans('', '', string.punctuation))

def fix_text(txt):
    return remove_punct(remove_non_ascii(txt).replace(" ", ""))
    
def highlight_file(pdfDoc, fileInfo, all, fileAnswerIdx, overallHighlightTEXT):
    foundPgNum = -1
    overallHighlight = False
    found = False
    
    highlightColors = [(14/255, 48/255, 0), (34/255, 224/255, 27/255), (151/255, 250/255, 147/255), (188/255, 245/255, 98/255), (252/255, 255/255, 161/255)]
    
    # iterate through pages in file
    for pgNum in range(len(pdfDoc)): 
        foundTxtLst = []
        page = pdfDoc[pgNum]
                
        contextLst = [line for line in fileInfo[1].split('\n') if line.strip() not in {"", "-", "/", "."}][1:-1]
                
        # highlight sections and open file
        blocks = page.get_text("dict")["blocks"]
                
        for blockIdx, block in enumerate(blocks): 
            
            # to highlight entire paragraph 
            paraHighlighted = False
            hangingLinesBBOX = []
            hangingLinesTEXT = []
            
            # look for text block by line
            if block['type'] == 0:
                for i, line in enumerate(block['lines']):
                    text = ""
                    bbox = []
                    lineHighlighted = False
                             
                    for span in line['spans']:
                        # extract bbox and text of line
                        bbox.append(span['bbox'])
                        text = text + " " + span['text']
                        
                    # check if text corresponds to context in contextLst
                    for context in contextLst: 
                        # if found:
                        #     print("\n context " + remove_punct(remove_non_ascii(context).replace(" ", "")))
                        #     print("\n text " + remove_punct(remove_non_ascii(text).replace(" ", "")) + "\n\n")
                            
                        #     if remove_punct(remove_non_ascii(context).replace(" ", "")) == remove_punct(remove_non_ascii(text).replace(" ", "")):
                        #         print("\nEQUAL\n")

                        #     if hangingLinesBBOX != []:
                        #         print("hangingLines " + page.get_textbox(hangingLinesBBOX[0]) + "\n\n")
                        
                        if (remove_punct(text.replace(" ", "")).isnumeric() and remove_non_ascii(text)==remove_non_ascii(context)) or (not(remove_punct(text.replace(" ", "")).isnumeric()) and (fix_text(context) == fix_text(text))):

                            print("\n context " + remove_punct(remove_non_ascii(context).replace(" ", "")))
                            print("\n text " + remove_punct(remove_non_ascii(text).replace(" ", "")) + "\n\n")
                            
                            # save pg number of where context is found
                            if not found:
                                foundPgNum = pgNum + 1
                                found = True

                            # highlight found context in page, using bbox coords of first and last bbox of line
                            if fix_text(text) not in overallHighlightTEXT:
                                if not all:
                                    page.add_highlight_annot((bbox[0][0], bbox[0][1], bbox[-1][2], bbox[-1][3]))
                                else:
                                    if fileAnswerIdx < len(highlightColors):
                                        hi = page.add_rect_annot((bbox[0][0], bbox[0][1], bbox[-1][2], bbox[-1][3])) 
                                        hi.set_colors(stroke=highlightColors[fileAnswerIdx])
                                        hi.update(opacity=0.2, fill_color=highlightColors[fileAnswerIdx])      
                                    else:
                                        hi = page.add_rect_annot((bbox[0][0], bbox[0][1], bbox[-1][2], bbox[-1][3]))
                                        hi.set_colors(stroke=highlightColors[-1])
                                        hi.update(opacity=0.2, fill_color=highlightColors[-1])
                                overallHighlightTEXT.append(fix_text(text))
                                    
                            # remove found context from context list
                            contextLst.remove(context)

                            paraHighlighted = True
                            lineHighlighted = True
                            overallHighlight = True
                                    
                            break

                    # if line is not highlighted than it is a hanging line
                    if not lineHighlighted:
                        hangingLinesBBOX.append((bbox[0][0], bbox[0][1], bbox[-1][2], bbox[-1][3]))
                        hangingLinesTEXT.append(text)
                        
                    # check if last character in text is period and we have yet to actually highlight paragraph - 
                    # case: previous paragraph that shouldn't be highlighted in block
                    if text.replace(" ", "")[-1] in [".", "!", "?", ":"] and paraHighlighted == False and overallHighlight == False:
                        hangingLinesBBOX = []
                        hangingLinesTEXT = []
                                
                    # check if last character in text is period - new paragraph 
                    # and we are no longer looking for context 
                    if text.replace(" ", "")[-1] in [".", "!", "?", ":"] and contextLst == [] and paraHighlighted == True:
                        paraHighlighted = False

                        for bboxHanging, textHanging in zip(hangingLinesBBOX, hangingLinesTEXT):
                            if fix_text(textHanging) not in overallHighlightTEXT:
                                if not all:
                                    page.add_highlight_annot(bboxHanging)
                                else:
                                    if fileAnswerIdx < len(highlightColors):
                                        hi = page.add_rect_annot(bboxHanging)
                                        hi.set_colors(stroke=highlightColors[fileAnswerIdx])
                                        hi.update(opacity=0.2, fill_color=highlightColors[fileAnswerIdx])
                                    else:
                                        hi = page.add_rect_annot(bboxHanging)
                                        hi.set_colors(stroke=highlightColors[-1])
                                        hi.update(opacity=0.2, fill_color=highlightColors[fileAnswerIdx])
                                overallHighlightTEXT.append(fix_text(textHanging))
                        hangingLinesBBOX = []
                                
                    # if last line in block check if block was partially highlighted then highlight hanging lines
                    if i == (len(block['lines'])-1) and paraHighlighted == True:
                            
                        for bbox, textHanging in zip(hangingLinesBBOX, hangingLinesTEXT):
                            if fix_text(textHanging) not in overallHighlightTEXT:
                                if not all:
                                    page.add_highlight_annot(bbox)
                                else:
                                    if fileAnswerIdx < len(highlightColors):
                                        hi = page.add_rect_annot(bbox)
                                        hi.set_colors(stroke=highlightColors[fileAnswerIdx])
                                        hi.update(opacity=0.2, fill_color=highlightColors[fileAnswerIdx])
                                    else:
                                        hi = page.add_rect_annot(bbox)
                                        hi.set_colors(stroke=highlightColors[-1])
                                        hi.update(opacity=0.2, fill_color=highlightColors[fileAnswerIdx])
                                overallHighlightTEXT.append(fix_text(textHanging))

                        # if there is a previous top hanging line highlight it - case: indented lines are considered 
                        # separate block - and top line of current block is highlighted
                        if blockIdx != 0 and blocks[blockIdx-1]['type'] == 0 and len(blocks[blockIdx-1]['lines']) == 1:
                            # get text of top line of current block

                            topLineCurrTEXT = ""

                            for span in blocks[blockIdx]['lines'][0]['spans']:
                                topLineCurrTEXT = topLineCurrTEXT + " " + span['text']

                            print("topLineCurrTEXT" + topLineCurrTEXT, fix_text(topLineCurrTEXT) in overallHighlightTEXT, "\n")
                            if fix_text(topLineCurrTEXT) in overallHighlightTEXT:
                                
                                topLineBBOX = []
                                topLineTEXT = ""
                                
                                for span in blocks[blockIdx-1]['lines'][0]['spans']:
                                    # extract bbox and text of top line
                                    topLineTEXT = topLineTEXT + " " + span['text']
                                    topLineBBOX.append(span['bbox'])

                                print("topLineTEXT" + topLineTEXT + "\n")
                                if fix_text(topLineTEXT) not in overallHighlightTEXT:
                                    if not all:
                                        page.add_highlight_annot((topLineBBOX[0][0], topLineBBOX[0][1], topLineBBOX[-1][2], topLineBBOX[-1][3]))
                                    else:
                                        if fileAnswerIdx < len(highlightColors):
                                            hi = page.add_rect_annot((topLineBBOX[0][0], topLineBBOX[0][1], topLineBBOX[-1][2], topLineBBOX[-1][3]))
                                            hi.set_colors(stroke=highlightColors[fileAnswerIdx])
                                            hi.update(opacity=0.2, fill_color=highlightColors[fileAnswerIdx])
                                        else:
                                            hi = page.add_rect_annot((topLineBBOX[0][0], topLineBBOX[0][1], topLineBBOX[-1][2], topLineBBOX[-1][3]))
                                            hi.set_colors(stroke=highlightColors[-1])
                                            hi.update(opacity=0.2, fill_color=highlightColors[fileAnswerIdx])
                                    overallHighlightTEXT.append(fix_text(topLineTEXT))
                            
    return foundPgNum, overallHighlightTEXT
    
@out.capture()
def show_results():
    # button to remove highlight and clear output 
    fileToClear = ''
    clearBtn = widgets.Button(description="Clear")
    prevFileName = "" 
    
    clearBtn.on_click(clear_btn_click)
    display(clearBtn)

    # iterate through all answers 
    for idx, answer in enumerate(answers):

        # check for a change in the filename for the answer
        if prevFileName != answer['fileName']:
            
            # button to highlight all context on file
            highlightAllButton = widgets.Button(description='Highlight All')

            def on_highlight_all_button_click(button, fileName=""):
                global fileToClear
                
                # clear on direct change
                clear_file()

                pdfDoc = fitz.open(fileName) 
                fileAnswerIdx = 0
                overallHighlightTEXT = []
                
                # iterate through answers to use only those with corresponding fileName
                for idx, answer in enumerate(answers):
                    
                    if answer['fileName'] == fileName:
                        
                        # highlight relevant sections
                        if fileAnswerIdx == 0:
                            foundPgNum, overallHighlightTEXT = highlight_file(pdfDoc, (answer['fileName'], answer['context']), True, fileAnswerIdx, overallHighlightTEXT)
                        else:
                            temp, overallHighlightTEXT = highlight_file(pdfDoc, (answer['fileName'], answer['context']), True, fileAnswerIdx, overallHighlightTEXT)
                            
                        fileAnswerIdx = fileAnswerIdx + 1
                        
                pdfDoc.save(fileName, incremental=True, encryption=fitz.PDF_ENCRYPT_KEEP)
                fileToClear = fileName
                        
                open_file(fileName, foundPgNum)

            highlightAllButton.on_click(functools.partial(on_highlight_all_button_click, fileName=answer['fileName']))
            
            display(highlightAllButton)

        prevFileName = answer['fileName']
        
        # display answer
        label = widgets.HTML(value="<style>p{word-wrap: break-word}</style> <p> File Name: " + answer['fileName'] + "</p>"  + "\n<p> Context: " + answer['context'] + "</p>")
        display(label)
        
        # button to highlight context on original PDF
        button = widgets.Button(description='Citation')
        
        def on_button_click(button, fileInfo=("", "")):
            global fileToClear
                        
            # if direct change from file to file, clear previous file
            clear_file()
            pdfDoc = fitz.open(fileInfo[0])        

            # highlight corresponding file
            foundPgNum, temp = highlight_file(pdfDoc, fileInfo, False, 0, [])
                            
            pdfDoc.save(fileInfo[0], incremental=True, encryption=fitz.PDF_ENCRYPT_KEEP)
            fileToClear = fileInfo[0]
            
            open_file(fileInfo[0], foundPgNum)

        button.on_click(functools.partial(on_button_click, fileInfo=(answer['fileName'], answer['context'])))
        display(button)
    

Text(value='', placeholder='Enter your question here')

Button(description='Search', style=ButtonStyle())

Output(layout=Layout(height='100%', width='100%'))

In [7]:
# OPTION #2 - INLINE GUI 

# import ipywidgets as widgets
# from IPython.display import display, IFrame, FileLink
# import fitz 
# import functools
# import string
# import os, sys, subprocess
# import webbrowser

# if answers:
#     def fix_text(context):
#         contextFix = (context).replace("-", "")
    
#         ascii_chars = set(string.printable)
#         contextFix = ''.join(filter(lambda x: x in ascii_chars, contextFix))

#         return contextFix
        
#     def preprocess_context(context):
#         # removing hyphen, newline and non-ascii characters
#         contextFix = fix_text(context)
    
#         contextFixLst = contextFix.split("\n")
    
#         contextFixLstWOWhite = []
#         for contextFix in contextFixLst:
#             if not all(ch==" " for ch in contextFix):
#                 contextFixLstWOWhite.append(contextFix)
                
#         # split content into a list of lines
#         return contextFixLstWOWhite
    
#     # to display relevant PDF
#     out = widgets.Output(layout={'border': '1px solid black', 'width':'400px', 'height':'600px'})
#     display(out)
    
#     # to display page number
#     outPgNum = widgets.Output(layout={'width':'300px', 'height':'50px'})
#     display(outPgNum)
    
#     # button to remove highlight and clear output 
#     fileToClear = ''
#     clearBtn = widgets.Button(description="Clear")
    
#     foundPgNum = -1
    
#     def clear_file():
#         if fileToClear != '':
#             pdfDoc = fitz.open(fileToClear)
        
#             # iterates pages and removes annotations generated by fitz
#             for pgNum in range(len(pdfDoc)):
#                 page = pdfDoc[pgNum]
        
#                 pdfDoc.xref_set_key(page.xref, "Annots", "[]")
            
#             pdfDoc.save(fileToClear, incremental=True, encryption=fitz.PDF_ENCRYPT_KEEP)
    
#             # clear output cell 
#             out.clear_output()
#             outPgNum.clear_output()
    
#     def clear_btn_click(button): clear_file();
#     clearBtn.on_click(clear_btn_click)
#     display(clearBtn)
    
#     # iterate through all answers 
#     for idx, answer in enumerate(answers):
        
#         # display answer
#         label = widgets.HTML(value="<style>p{word-wrap: break-word}</style> <p> File Name: " + answer['fileName'] + "</p>" + 
#                              "\n<p> Answer: " + answer['answer'] + "</p>")
#         display(label)
    
#         # button to highlight context on original PDF
#         button = widgets.Button(description='Context')
    
#         def on_button_click(button, fileInfo=("", "")):
#             global foundPgNum
#             global fileToClear
            
#             # if direct change from file to file, clear previous file
#             clear_file()
            
#             pdfDoc = fitz.open(fileInfo[0])        
#             found = False                    
                
#             # iterate through pages in file
#             for pgNum in range(len(pdfDoc)):
            
#                 page = pdfDoc[pgNum]            
#                 contextLst = preprocess_context(fileInfo[1])
                
#                 found = False
                
#                 # highlight sections and open file
                
#                 blocks = page.get_text("dict")["blocks"]
                
#                 for block in blocks: 
                    
#                     # look for text block by line
#                     if block['type'] == 0:
                        
#                         for line in block['lines']:
#                             for span in line['spans']:
                                
#                                 # extract bbox and text of line
#                                 bbox = span['bbox']
#                                 text = span['text']

#                                 # check if text corresponds to context in contextLst
#                                 for context in contextLst:
    
#                                     if context.replace(" ", "") == fix_text(text).replace(" ", ""):
#                                         # save pg number of where context is found
#                                         if not found:
#                                             foundPgNum = pgNum + 1
#                                             found = True
                                            
#                                         # highlight found context in page
#                                         page.add_highlight_annot(bbox)
            
#             pdfDoc.save(fileInfo[0], incremental=True, encryption=fitz.PDF_ENCRYPT_KEEP)
#             fileToClear = fileInfo[0]
            
#             # clear output cell and render new file
#             out.clear_output()
#             @out.capture()
#             def render():
#                 display(IFrame(src=fileInfo[0],width=400, height=600))
#             render()
    
#             outPgNum.clear_output()
#             @outPgNum.capture()
#             def renderPgNum():
#                 display(HTML("<p> File Name: " + fileInfo[0] + "</p>"))
#                 display(HTML("<p> Page Number: " + str(foundPgNum) + "</p>"))
#             renderPgNum()
            
#         button.on_click(functools.partial(on_button_click, fileInfo=(answer['fileName'], answer['context'])))
#         display(button)


In [8]:
# TEST CELL

# def open_file(filename, pgNum):
#     """ UNUSED """
#     currDir = os.getcwdb()
    
#     # opens file in default browser at specific page, NOTE: assumes that file is in current directory and Chrome is installed
#     fileLoc = "file:///" + str(currDir)[2:][:-1] + "/" + filename + "#page=" + str(pgNum)
#     (webbrowser.get(using='chrome')).open(fileLoc)  

#     # alternative method - not specific to page
#     # fileLoc = "file:///" + str(currDir)[2:][:-1] + "/" + filename + "#page=" + str(pgNum)
#     # webbrowser.open(fileLoc)

In [9]:
# TEST CELL

# import ipywidgets as widgets
# from IPython.display import display, IFrame

# out = widgets.Output(layout={'border': '1px solid black', 'width':'400px', 'height':'600px'})
# display(out)

# @out.capture()
# def render():
#     display(IFrame(src="sample-pdf-cs-1.pdf",width=400, height=600))

# render()

# but = widgets.Button(description='test')
# def test_click(button):
#     out.clear_output()
# but.on_click(test_click)
# display(but)

In [10]:
# TEST CELL

# tab = widgets.Tab(children=[widgets.IntSlider(), widgets.Text()], titles=('Slider', 'Text'))
# test = widgets.Button(description='test')

# def test_click(button):
    
#     tab.titles = tab.titles + tuple(["test"])
#     print(tab.titles)
#     tab.children = tab.children + tuple([widgets.Text()])
    
# test.on_click(test_click)
# display(tab, test)

In [11]:
# TEST CELL

# from IPython.display import IFrame
# fra = IFrame("sample-pdf-cs-1.pdf", width=600, height=400)
# display(fra)
# fra.layout.visibility = 'hidden'

# img = widgets.Image(value=file_upload.value[0].content.tobytes(), width = 400, height=600)
# display(img)
# widgets.HTML(
#     value='''<embed src="sample-pdf-cs-1.pdf#page=2" type="application/pdf" width="100%" height="600px" />''',
#     placeholder='Some HTML',
#     description='Some HTML',
# )

# HTML(''' <embed src="sample-pdf-cs-1.pdf#page=2" type="application/pdf" width="100%" height="600px" /> ''')
# HTML('''<object
# 	data="sample-pdf-cs-1.pdf#page=2"
# 	type="application/pdf"
# 	width="600"
# 	height="400">
# 	<p>
# 		Your browser does not support PDFs.
# 		<a href="sample-pdf-cs-1.pdf">Download the PDF</a>.
# 	</p>
# </object>''')

# h = widgets.HTML('''<iframe src="sample-pdf-cs-1.pdf" width=400 height=600></iframe>''')
# display(h)

# HTML('''<p>
# 	Open a PDF file
# 	<a href="sample-pdf-cs-1.pdf#page=2">example</a>
# </p>''')

# from wand.image import Image as WImage
# img = WImage(filename='sample-pdf-cs-1.pdf[2]')
# img